# Import Librairies 

In [30]:
# manipulate data 
import seaborn as sns
import pandas as pd
import numpy as np
#data vizualisation
import matplotlib.pyplot as plt
# from wordcloud import WordCloud, STOPWORDS
import io
import pickle


# pre traitment & model
from keras.losses import categorical_crossentropy
from keras.models import Sequential, load_model
from keras.preprocessing.text import Tokenizer
from keras.utils import to_categorical
from keras_preprocessing.sequence import pad_sequences
from keras.models import Model
from keras.layers import GRU, Input, Dense, TimeDistributed, Activation, RepeatVector, Bidirectional, Dropout, LSTM
from tensorflow.keras.layers import Embedding
from keras.optimizers import Adam
from keras.losses import categorical_crossentropy

# Upload data

In [2]:
french_sentences = open("./data/small_vocab_fr.txt", encoding="utf-8").read().split('\n')
english_sentences = open("./data/small_vocab_en.txt", encoding="utf-8").read().split('\n')
data_fr = pd.read_csv('./data/small_vocab_fr.txt', sep = "\t", header = None)
data_en = pd.read_csv('./data/small_vocab_en.txt', sep = "\t", header = None)

# tokenisation 
Pour qu'un réseau neuronal puisse prédire sur des données textuelles, celles-ci doivent d'abord être transformées en données qu'il peut comprendre. Des données textuelles comme " chien " sont une séquence d'encodages de caractères ASCII. Comme un réseau neuronal est une série d'opérations de multiplication et d'addition, les données d'entrée doivent être des nombres.

Nous pouvons transformer chaque caractère en un nombre ou chaque mot en un nombre. On parle alors d'identifiants de caractères et de mots, respectivement. Les identifiants des caractères sont utilisés pour les modèles de niveau caractère qui génèrent des prédictions de texte pour chaque caractère. Un modèle au niveau du mot utilise des identifiants de mot qui génèrent des prédictions de texte pour chaque mot. Les modèles au niveau des mots ont tendance à mieux apprendre, car ils sont moins complexes, nous les utiliserons donc.

Transformez chaque phrase en une séquence d'identifiants de mots en utilisant la fonction Tokenizer de Keras. Utilisez cette fonction pour tokeniser les phrases anglaises et françaises dans la cellule ci-dessous.

L'exécution de la cellule lancera tokenize sur des données d'échantillon et montrera la sortie pour le débogage.

In [3]:
def tokenize(x):
    """
    Tokenize x
    :param x: List of sentences/strings to be tokenized
    :return: Tuple of (tokenized x data, tokenizer used to tokenize x)
    """
    print(x)
    x_tk = Tokenizer(char_level = False)
    x_tk.fit_on_texts(x)
    return x_tk.texts_to_sequences(x), x_tk

# Tokenize Example output
text_sentences = [
    'The quick brown fox jumps over the lazy dog .',
    'By Jove , my quick study of lexicography won a prize .',
    'This is a short sentence .']
text_tokenized, text_tokenizer = tokenize(text_sentences)
print(text_tokenizer.word_index)
print()
for sample_i, (sent, token_sent) in enumerate(zip(text_sentences, text_tokenized)):
    print('Sequence {} in x'.format(sample_i + 1))
    print('  Input:  {}'.format(sent))
    print('  Output: {}'.format(token_sent))

['The quick brown fox jumps over the lazy dog .', 'By Jove , my quick study of lexicography won a prize .', 'This is a short sentence .']
{'the': 1, 'quick': 2, 'a': 3, 'brown': 4, 'fox': 5, 'jumps': 6, 'over': 7, 'lazy': 8, 'dog': 9, 'by': 10, 'jove': 11, 'my': 12, 'study': 13, 'of': 14, 'lexicography': 15, 'won': 16, 'prize': 17, 'this': 18, 'is': 19, 'short': 20, 'sentence': 21}

Sequence 1 in x
  Input:  The quick brown fox jumps over the lazy dog .
  Output: [1, 2, 4, 5, 6, 7, 1, 8, 9]
Sequence 2 in x
  Input:  By Jove , my quick study of lexicography won a prize .
  Output: [10, 11, 12, 2, 13, 14, 15, 16, 3, 17]
Sequence 3 in x
  Input:  This is a short sentence .
  Output: [18, 19, 3, 20, 21]


# padding 
Lorsque l'on regroupe les séquences d'identifiants de mots, chaque séquence doit être de la même longueur. Comme la longueur des phrases est dynamique, nous pouvons ajouter un remplissage à la fin des séquences pour qu'elles aient la même longueur.

Assurez-vous que toutes les séquences anglaises ont la même longueur et que toutes les séquences françaises ont la même longueur en ajoutant du remplissage à la fin de chaque séquence en utilisant la fonction pad_sequences de Keras.


In [4]:
def pad(x, length=None):
    """
    Pad x
    :param x: List of sequences.
    :param length: Length to pad the sequence to.  If None, use length of longest sequence in x.
    :return: Padded numpy array of sequences
    """
    # TODO: Implement
    if length is None:
        length = max([len(sentence) for sentence in x])
    return pad_sequences(x, maxlen = length, padding = 'post')

# Pad Tokenized output
test_pad = pad(text_tokenized)
for sample_i, (token_sent, pad_sent) in enumerate(zip(text_tokenized, test_pad)):
    print('Sequence {} in x'.format(sample_i + 1))
    print('  Input:  {}'.format(np.array(token_sent)))
    print('  Output: {}'.format(pad_sent))

Sequence 1 in x
  Input:  [1 2 4 5 6 7 1 8 9]
  Output: [1 2 4 5 6 7 1 8 9 0]
Sequence 2 in x
  Input:  [10 11 12  2 13 14 15 16  3 17]
  Output: [10 11 12  2 13 14 15 16  3 17]
Sequence 3 in x
  Input:  [18 19  3 20 21]
  Output: [18 19  3 20 21  0  0  0  0  0]


# pre processing pipeline

Your focus for this project is to build neural network architecture, so we won't ask you to create a preprocess pipeline. Instead, we've provided you with the implementation of the preprocess function.

In [5]:
def preprocess(x, y):
    """
    Preprocess x and y
    :param x: Feature List of sentences
    :param y: Label List of sentences
    :return: Tuple of (Preprocessed x, Preprocessed y, x tokenizer, y tokenizer)
    """
    preprocess_x, x_tk = tokenize(x)
    preprocess_y, y_tk = tokenize(y)

    preprocess_x = pad(preprocess_x)
    preprocess_y = pad(preprocess_y)

    # Keras's sparse_categorical_crossentropy function requires the labels to be in 3 dimensions
    preprocess_y = preprocess_y.reshape(*preprocess_y.shape, 1)

    return preprocess_x, preprocess_y, x_tk, y_tk

preproc_english_sentences, preproc_french_sentences, english_tokenizer, french_tokenizer =\
    preprocess(english_sentences, french_sentences)
    
max_english_sequence_length = preproc_english_sentences.shape[1]
max_french_sequence_length = preproc_french_sentences.shape[1]
english_vocab_size = len(english_tokenizer.word_index)
french_vocab_size = len(french_tokenizer.word_index)

print('Data Preprocessed')
print("Max English sentence length:", max_english_sequence_length)
print("Max French sentence length:", max_french_sequence_length)
print("English vocabulary size:", english_vocab_size)
print("French vocabulary size:", french_vocab_size)


IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



Data Preprocessed
Max English sentence length: 15
Max French sentence length: 21
English vocabulary size: 199
French vocabulary size: 344


In [6]:
# saving tokenizer
with open('french_tokenizer.pickle', 'wb') as handle:
    pickle.dump(french_tokenizer, handle, protocol=pickle.HIGHEST_PROTOCOL)
# english_tokenizer

In [7]:
# loading tokenizer
with open('english_tokenizer.pickle', 'rb') as handle:
    tokenizer = pickle.load(handle)

print(tokenizer)

# Model
De l'identité au texte
Le réseau neuronal va traduire l'entrée en mots ids, ce qui n'est pas la forme finale que nous voulons. Nous voulons la traduction française. La fonction logits_to_text fait le lien entre les logits du réseau neuronal et la traduction française. Vous utiliserez cette fonction pour mieux comprendre la sortie du réseau neuronal.

In [8]:
def logits_to_text(logits, tokenizer):
    """
    Turn logits from a neural network into text using the tokenizer
    :param logits: Logits from a neural network
    :param tokenizer: Keras Tokenizer fit on the labels
    :return: String that represents the text of the logits
    """
    index_to_words = {id: word for word, id in tokenizer.word_index.items()}
    index_to_words[0] = ''

    return ' '.join([index_to_words[prediction] for prediction in np.argmax(logits, 1)])

print('`logits_to_text` function loaded.')

`logits_to_text` function loaded.


In [51]:
def model_final(input_shape, output_sequence_length, english_vocab_size, french_vocab_size):
    """
    Build and train a model that incorporates embedding, encoder-decoder, and bidirectional RNN on x and y
    :param input_shape: Tuple of input shape
    :param output_sequence_length: Length of output sequence
    :param english_vocab_size: Number of unique English words in the dataset
    :param french_vocab_size: Number of unique French words in the dataset
    :return: Keras model built, but not trained
    """
    # TODO: Implement

    # Hyperparameters
    learning_rate = 0.003
    
#     # Build the layers    
#     model = Sequential()
    
#     # Embedding
#     model.add(Embedding(english_vocab_size, 128, input_length=input_shape[1],
#                          input_shape=input_shape[1:]))
#     # Encoder
#     model.add(Bidirectional(GRU(128)))
#     model.add(RepeatVector(output_sequence_length))
#     # Decoder
#     model.add(Bidirectional(GRU(128, return_sequences=True)))
#     model.add(TimeDistributed(Dense(512, activation='relu')))
#     model.add(Dropout(0.5))
#     model.add(TimeDistributed(Dense(french_vocab_size, activation='softmax')))
    
    
    model = Sequential()
    # Add an Embedding layer expecting input vocab of size 1000, and
    # output embedding dimension of size 64.
    model.add(Embedding(english_vocab_size, 
                                  128, 
                                  input_length=input_shape[1],
                                  input_shape=input_shape[1:]))

    # Add a LSTM layer with 128 internal units.
    model.add(LSTM(128))

    # Add a Dense layer with 10 units.
#     model.add(Dense(21))
# repeatvector repeats the input for the desired number of times to change 2D-array to 3D array
    model.add(RepeatVector(max_french_sequence_length))
    model.add(LSTM(256, return_sequences = True))
    model.add(TimeDistributed(Dense(french_vocab_size, activation = 'softmax')))

    model.summary()
    
    
    model.compile(loss=categorical_crossentropy,
                  optimizer=Adam(learning_rate),
                  metrics=['accuracy'])    
    return model

In [52]:
# fit model
model = model_final(preproc_english_sentences.shape,
                    preproc_french_sentences.shape[1],
                    len(english_tokenizer.word_index)+1,
                    len(french_tokenizer.word_index)+1)

Model: "sequential_13"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_12 (Embedding)    (None, 15, 128)           25600     
                                                                 
 lstm_11 (LSTM)              (None, 128)               131584    
                                                                 
 repeat_vector_1 (RepeatVect  (None, 21, 128)          0         
 or)                                                             
                                                                 
 lstm_12 (LSTM)              (None, 21, 256)           394240    
                                                                 
 time_distributed_3 (TimeDis  (None, 21, 345)          88665     
 tributed)                                                       
                                                                 
Total params: 640,089
Trainable params: 640,089
Non-t

In [54]:
preproc_english_sentences.shape, preproc_french_sentences.shape

((137861, 15), (137861, 21, 1))

In [53]:
history = model.fit(preproc_english_sentences, preproc_french_sentences, batch_size=520, epochs=25, validation_split=0.2)

Epoch 1/25


ValueError: in user code:

    File "C:\Users\mathi\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\keras\engine\training.py", line 1160, in train_function  *
        return step_function(self, iterator)
    File "C:\Users\mathi\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\keras\engine\training.py", line 1146, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "C:\Users\mathi\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\keras\engine\training.py", line 1135, in run_step  **
        outputs = model.train_step(data)
    File "C:\Users\mathi\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\keras\engine\training.py", line 994, in train_s

In [ ]:
# plot accuracy & loss metrics
def plot_metrics(metrics, history):
    loss_train = history.history[metrics]
    loss_val = history.history['val_'+metrics]
    epochs = range(1,26)
    label_train = 'Training' + metrics
    label_val = 'Validation' + metrics
    plt.plot(epochs, loss_train, 'g', label=label_train)
    plt.plot(epochs, loss_val, 'b', label=label_val)
    plt.title('Training and Validation loss')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.legend()
    plt.show()

In [ ]:
plot_metrics('accuracy', history)

In [ ]:
plot_metrics('loss', history)

In [ ]:
def preprocess_predict():
    
    y_id_to_word = {value: key for key, value in french_tokenizer.word_index.items()}
    y_id_to_word[0] = ''

    sentence = 'india is relaxing during may but it is usually busy in winter'
    
    sentence = [english_tokenizer.word_index[word] for word in sentence.split()]
    
    sentence = pad_sequences([sentence], maxlen=preproc_english_sentences.shape[-1], padding='post')
    
    sentences = np.array([sentence[0], preproc_english_sentences[0]])
    
    predictions = model.predict(sentences, len(sentences))
    print(' '.join([y_id_to_word[np.argmax(x)] for x in predictions[0]]))

In [ ]:
preprocess_predict()

In [ ]:
fdfddfdfdf

In [ ]:
# prediction
y_id_to_word = {value: key for key, value in y_tk.word_index.items()}
y_id_to_word[0] = ''

sentence = 'new jersey is sometines beautiful'
sentence = [x_tk.word_index[word] for word in sentence.split()]
sentence = pad_sequences([sentence], maxlen=x.shape[-1], padding='post')
sentences = np.array([sentence[0], x[0]])
predictions = model.predict(sentences, len(sentences))

print('Sample 1:')
print(' '.join([y_id_to_word[np.argmax(x)] for x in predictions[0]]))
print('Sample 2:')
print(' '.join([y_id_to_word[np.argmax(x)] for x in predictions[1]]))
print(' '.join([y_id_to_word[np.max(x)] for x in y[0]]))

In [ ]:
def final_predictions(x, y, x_tk, y_tk):
    """
    Gets predictions using the final model
    :param x: Preprocessed English data
    :param y: Preprocessed French data
    :param x_tk: English tokenizer
    :param y_tk: French tokenizer
    """
    # TODO: Train neural network using model_final
    print(x.shape, y.shape[1], len(x_tk.word_index)+1, len(y_tk.word_index)+1)
    
    model = model_final(x.shape,y.shape[1],
                        len(x_tk.word_index)+1,
                        len(y_tk.word_index)+1)
    model.summary()
    history = model.fit(x, y, batch_size=520, epochs=25, validation_split=0.2)
    model.save("model.h5")
    # DON'T EDIT ANYTHING BELOW THIS LINE
    y_id_to_word = {value: key for key, value in y_tk.word_index.items()}
    y_id_to_word[0] = ''

    sentence = 'new jersey is sometines beautiful'
    sentence = [x_tk.word_index[word] for word in sentence.split()]
    sentence = pad_sequences([sentence], maxlen=x.shape[-1], padding='post')
    sentences = np.array([sentence[0], x[0]])
    predictions = model.predict(sentences, len(sentences))

    print('Sample 1:')
    print(' '.join([y_id_to_word[np.argmax(x)] for x in predictions[0]]))
    print('Sample 2:')
    print(' '.join([y_id_to_word[np.argmax(x)] for x in predictions[1]]))
    print(' '.join([y_id_to_word[np.max(x)] for x in y[0]]))


final_predictions(preproc_english_sentences, preproc_french_sentences, english_tokenizer, french_tokenizer)

In [ ]:
model = load_model('model/model.h5')

In [ ]:
# DON'T EDIT ANYTHING BELOW THIS LINE

def preprocess_predict():
    
    y_id_to_word = {value: key for key, value in french_tokenizer.word_index.items()}
    y_id_to_word[0] = ''

    sentence = 'india is relaxing during may but it is usually busy in winter'
    
    sentence = [english_tokenizer.word_index[word] for word in sentence.split()]
    
    sentence = pad_sequences([sentence], maxlen=preproc_english_sentences.shape[-1], padding='post')
    
    sentences = np.array([sentence[0], preproc_english_sentences[0]])
    
    predictions = model.predict(sentences, len(sentences))
    print(' '.join([y_id_to_word[np.argmax(x)] for x in predictions[0]]))
#     return predictions


In [ ]:
%%timeit -r 1 -n 1
res = preprocess_predict()
print(res)

In [ ]:
print(res)

# Vizualise accuracy & loss

In [ ]:
loss_train = history.history['loss']
loss_val = history.history['val_loss']
epochs = range(1,26)b
plt.plot(epochs, loss_train, 'g', label='Training loss')
plt.plot(epochs, loss_val, 'b', label='validation loss')
plt.title('Training and Validation loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()

In [ ]:
loss_train = history.history['accuracy']
loss_val = history.history['val_accuracy']
epochs = range(1,26)
plt.plot(epochs, loss_train, 'g', label='Training loss')
plt.plot(epochs, loss_val, 'b', label='validation loss')
plt.title('Training and Validation loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()

# model save


In [ ]:
history.save('model')